#   MTCNN Face Detection - Implantação

*   Este componente utiliza a biblioteca [facenet-pytorch](https://github.com/timesler/facenet-pytorch), a qual disponibiliza o algorítimo [MTCNN](https://arxiv.org/abs/1604.02878). 

* O MTCNN possui a performance estado da arte nos benchmarks [FDDB](http://vis-www.cs.umass.edu/fddb/) e [WIDER FACE](http://shuoyang1213.me/WIDERFACE/)

*   Melhores explicações são encontradas neste [artigo do kaggle](https://www.kaggle.com/timesler/guide-to-mtcnn-in-facenet-pytorch)


### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
!pip install facenet-pytorch --quiet

In [10]:
%%writefile Model.py
from typing import List, Iterable, Dict, Union

import joblib
import pandas as pd
import numpy as np
import cv2
from mtcnn import MTCNN_Model

class Model(object):
    
    def __init__(self):
        self.loaded = False

    def load(self):
        
        # Following links explain why to use load() method insted of __init__()

        # Issue associated with sheldon on __init__
        # https://github.com/SeldonIO/seldon-core/issues/2616

        # Solution for the case
        # https://docs.seldon.io/projects/seldon-core/en/latest/python/python_component.html#gunicorn-and-load

        artifacts = joblib.load("/tmp/data/mtcnn.joblib")
        self.model_parameters = artifacts["model_parameters"]
        self.inference_parameters = artifacts["inference_parameters"]
        
        # Initializate model
        self.model = MTCNN_Model(self.model_parameters, self.inference_parameters)

        # Health check validation
        random_img = np.random.randint(0, 255, size=(100, 100, 3), dtype=np.uint8)
        _, _ = self.model.predict(random_img)

        self.loaded = True
        
    def class_names(self) -> List:
        
        return ['x_min', 'y_min', 'x_max', 'y_max', 'probability']
        
    def format_result(self, bboxes: Union[np.ndarray, List], probs: Union[np.ndarray, List]) -> np.ndarray:
        
        results = []
        
        for bbox_id, prob in enumerate(probs):
            
            # Check if has found a bbox for the image
            if prob is None:
                res = np.full((5), None)
            
            # Extend to an array the 4coords and prob
            else:
                bbox = np.array(bboxes[bbox_id])
                prob = np.array([prob])

                res = np.concatenate((bbox, prob), axis=0)
                
            results.append(res)
        
        return np.array(results).astype(float)
    
    def predict(self, X: np.ndarray, feature_names: Iterable[str], meta: Dict = None) -> Union[np.ndarray, List, str, bytes]:

        # First time load model
        if not self.loaded:
            self.load()

        # Check if data is a bytes
        if isinstance(X, bytes):
            im_bytes = X # Get image bytes
        
        # If not, should be a list or ndarray
        else:
            # Garantee is a ndarray
            X = np.array(X)
            
            # Seek for extra dimension
            if len(X.shape) == 2:
                im_bytes = X[0,0] # Get image bytes
            
            else:
                im_bytes = X[0] # Get image bytes

        # Preprocess img bytes to img_arr
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)
        img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Predict results
        bboxes, probs = self.model.predict(img)

        # Remove batch dimension
        bboxes, probs = np.squeeze(bboxes, 0), np.squeeze(probs, 0)
        
        # Compile results
        results = self.format_result(bboxes, probs)
        
        return results

Overwriting Model.py
